In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
csv_path = "final.csv"
image_path = "./images"
batch_size = 256
epochs = 200
learning_rate = 1e-3
seed = 42

In [3]:
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [4]:
class ImageDataset(Dataset):
    def __init__(self, csv_path, image_path, image_transform=transforms.Compose([transforms.ToTensor()])):
        super(ImageDataset).__init__()
        csv = pd.read_csv(csv_path)
        csv_np = csv.to_numpy()
        self.images = []
        for Id in csv['Id']:
            Id = str(Id)
            image = Image.open(image_path+'/'+Id+'.jpg')
            self.images.append(image_transform(image))
            image.close()
        
        self.target = torch.from_numpy(np.array(csv_np[:,[12]], dtype="float")).float()
    
    
    def __getitem__(self,idx):
        return self.images[idx], self.target[idx]
    
    
    def __len__(self):
        return len(self.images)

In [5]:
data_length = len(pd.read_csv(csv_path))
train_length = int(data_length * 0.6)
test_length = int(data_length * 0.2)
val_length = data_length - train_length - test_length

image_transform = transforms.Compose([transforms.ToTensor()
                                     ])

train_dataset = ImageDataset(csv_path, image_path, image_transform)
train_dataset, test_dataset = random_split(train_dataset, [train_length, test_length+val_length])
test_dataset, val_dataset = random_split(test_dataset, [test_length, val_length])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle = True)

In [6]:
class ImageNet(nn.Module):
    def __init__(self):
        super(ImageNet, self).__init__()
        self.resnet18 = torchvision.models.resnet18(pretrained=False)
        self.resnet18.fc = nn.Sequential(
        nn.Linear(512, 4096),
        nn.BatchNorm1d(4096),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(4096, 2048),
        nn.BatchNorm1d(2048),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(2048, 1024),
        nn.BatchNorm1d(1024),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(1024, 1)
        )
        self.resnet18.to(device)
    
    def forward(self, x):
        return self.resnet18(x)

In [7]:
model = ImageNet().to(device)

criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
for epoch in range(epochs):
    model.train()
    criterion.train()
    
    avg_loss = 0

    for X, Y in train_loader:
        X = X.to(device)
        Y = Y.to(device)

        model.zero_grad()
        prediction = model(X)
        loss = torch.sqrt(criterion(prediction, Y)).to(device)
        loss.backward()
        optimizer.step()
        avg_loss += loss / len(train_loader)
    print(f'[Epoch: {epoch+1:>2}] Average loss: {avg_loss:.4f}, ', end='')
    
    model.eval()
    criterion.eval()
    with torch.no_grad():
        val_avg_loss = 0.
        for X_val, Y_val in val_loader:
            X_val = X_val.to(device)
            Y_val = Y_val.to(device)
            val_prediction = model(X_val)
            val_loss = torch.sqrt(criterion(val_prediction, Y_val)).to(device)
            val_avg_loss += val_loss / len(val_loader)
        
        print(f"val_loss: {val_avg_loss:.4f}")

[Epoch:  1] Average loss: 4572.0889, val_loss: 4863.4121
[Epoch:  2] Average loss: 4566.9805, val_loss: 4890.8384
[Epoch:  3] Average loss: 4503.0176, val_loss: 4823.6953
[Epoch:  4] Average loss: 4574.6846, val_loss: 5000.8096
[Epoch:  5] Average loss: 4517.0200, val_loss: 4512.5684
[Epoch:  6] Average loss: 4319.8184, val_loss: 4100.5010
[Epoch:  7] Average loss: 4435.8877, val_loss: 4912.2930
[Epoch:  8] Average loss: 4724.2705, val_loss: 4817.5645
[Epoch:  9] Average loss: 6200.4189, val_loss: 4711.5063
[Epoch: 10] Average loss: 4397.5781, val_loss: 4624.6470
[Epoch: 11] Average loss: 4538.6699, val_loss: 4875.1357
[Epoch: 12] Average loss: 4549.2339, val_loss: 4466.2437
[Epoch: 13] Average loss: 4505.3730, val_loss: 4603.8525
[Epoch: 14] Average loss: 4466.6616, val_loss: 4061.5317
[Epoch: 15] Average loss: 4573.0615, val_loss: 4203.7974
[Epoch: 16] Average loss: 4420.8696, val_loss: 4396.5713
[Epoch: 17] Average loss: 4286.6011, val_loss: 4777.4912
[Epoch: 18] Average loss: 4336.

[Epoch: 144] Average loss: 1345.7349, val_loss: 3559.2500
[Epoch: 145] Average loss: 1333.1497, val_loss: 6081.1035
[Epoch: 146] Average loss: 1463.0696, val_loss: 5338.1206
[Epoch: 147] Average loss: 1549.3533, val_loss: 4041.2915
[Epoch: 148] Average loss: 1225.0027, val_loss: 4245.7891
[Epoch: 149] Average loss: 1520.1891, val_loss: 3856.6831
[Epoch: 150] Average loss: 1234.0702, val_loss: 3592.2451
[Epoch: 151] Average loss: 1235.5819, val_loss: 3750.9272
[Epoch: 152] Average loss: 1386.1284, val_loss: 4006.7271
[Epoch: 153] Average loss: 1150.5261, val_loss: 3615.2449
[Epoch: 154] Average loss: 2169.3391, val_loss: 4234.1323
[Epoch: 155] Average loss: 1096.9482, val_loss: 3972.5884
[Epoch: 156] Average loss: 1156.2188, val_loss: 4248.6133
[Epoch: 157] Average loss: 1183.9086, val_loss: 3609.0383
[Epoch: 158] Average loss: 1215.7528, val_loss: 4089.7185
[Epoch: 159] Average loss: 1358.9457, val_loss: 4142.6758
[Epoch: 160] Average loss: 1252.9128, val_loss: 3450.8843
[Epoch: 161] A

In [9]:
model.eval()
criterion.eval()
ss_tot = 0
ss_res = 0
with torch.no_grad():
    for X_test, Y_test in test_loader:
        X_test = X_test.to(device)
        Y_test = Y_test.to(device)
        prediction = model(X_test)
        mean = torch.mean(Y_test)
        ss_tot += torch.sum((Y_test - mean) ** 2)
        ss_res += torch.sum((Y_test - prediction) ** 2)
    
    accuracy = 1 - ss_res/ss_tot
    print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 17.74%
